In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-w3p2/train_data.csv
/kaggle/input/2022-ai-w3p2/test_data.csv
/kaggle/input/2022-ai-w3p2/sample_submit.csv


In [2]:
import random
import torch.optim as optim
import torch

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
from sklearn.metrics import mean_absolute_error

In [4]:
train = pd.read_csv('/kaggle/input/2022-ai-w3p2/train_data.csv')
test = pd.read_csv('/kaggle/input/2022-ai-w3p2/test_data.csv')
submit = pd.read_csv('/kaggle/input/2022-ai-w3p2/sample_submit.csv')

In [5]:
n = len(test)

In [6]:
train

,ID,company,model,year,transmission,mileage,fueltype,tax,mpg,engineSize,price
0,0,2,Focus,2019,1,3764,4,145,34.5,2.3,26299
1,1,1,I10,2015,1,27056,4,20,60.1,1.0,5495
2,2,3,A3,2019,3,3375,4,145,34.5,2.0,32400
3,3,3,A6,2019,0,5743,0,145,47.9,2.0,29990
4,4,2,Fiesta,2015,1,32782,4,0,65.7,1.0,7250
...,...,...,...,...,...,...,...,...,...,...,...
17080,17080,0,1 Series,2018,0,4666,4,145,32.5,3.0,23499
17081,17081,3,A4,2020,3,22601,4,145,39.2,2.0,32481
17082,17082,4,Yaris,2018,1,20312,4,145,47.1,1.5,10950
17083,17083,2,Focus,2018,1,18695,4,145,58.9,1.0,13295


In [7]:
train.columns

Index(['ID', 'company', 'model', 'year', 'transmission', 'mileage', 'fueltype',
       'tax', 'mpg', 'engineSize', 'price'],
      dtype='object')

In [8]:
x_train = np.array(train[['year', 'transmission', 'mileage', 'fueltype', 'tax', 'mpg', 'engineSize']])
y_train = np.array(train['price'])
y_train = y_train.reshape(-1,1)
x_test = np.array(test[['year', 'transmission', 'mileage', 'fueltype', 'tax', 'mpg', 'engineSize']])

In [9]:
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)
x_test = torch.FloatTensor(x_test)

In [10]:
print(x_train)
print(y_train)
print(x_test)

tensor([[2.0190e+03, 1.0000e+00, 3.7640e+03,  ..., 1.4500e+02, 3.4500e+01,
         2.3000e+00],
        [2.0150e+03, 1.0000e+00, 2.7056e+04,  ..., 2.0000e+01, 6.0100e+01,
         1.0000e+00],
        [2.0190e+03, 3.0000e+00, 3.3750e+03,  ..., 1.4500e+02, 3.4500e+01,
         2.0000e+00],
        ...,
        [2.0180e+03, 1.0000e+00, 2.0312e+04,  ..., 1.4500e+02, 4.7100e+01,
         1.5000e+00],
        [2.0180e+03, 1.0000e+00, 1.8695e+04,  ..., 1.4500e+02, 5.8900e+01,
         1.0000e+00],
        [2.0140e+03, 1.0000e+00, 3.7812e+04,  ..., 0.0000e+00, 7.6400e+01,
         1.5000e+00]])
tensor([[26299.],
        [ 5495.],
        [32400.],
        ...,
        [10950.],
        [13295.],
        [ 7299.]])
tensor([[2.0170e+03, 1.0000e+00, 1.0150e+04,  ..., 1.4500e+02, 5.7700e+01,
         1.2000e+00],
        [2.0130e+03, 1.0000e+00, 5.0216e+04,  ..., 1.4500e+02, 5.3300e+01,
         2.0000e+00],
        [2.0160e+03, 1.0000e+00, 3.4189e+04,  ..., 2.0000e+02, 4.7900e+01,
         2.00

In [11]:
W = torch.zeros((7,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [12]:
optimizer = optim.SGD([W, b], lr=0.000013)
nb_epochs = 500000

for epoch in range(nb_epochs + 1):
    hypothesis = x_train.matmul(W)+b
    cost = torch.abs(hypothesis - y_train).sum()/n
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 10000 == 0:
        print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
            epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
        )) 

Epoch    0/500000 hypothesis: tensor([0., 0., 0.,  ..., 0., 0., 0.]) Cost: 17073.593750
Epoch 10000/500000 hypothesis: tensor([17921.6895, 14304.1055, 17975.5352,  ..., 15621.5811, 15843.1543,
        12748.0176]) Cost: 6332.573730
Epoch 20000/500000 hypothesis: tensor([17990.4668, 14247.5508, 18041.7969,  ..., 15793.6133, 15999.6191,
        12712.8350]) Cost: 6297.756836
Epoch 30000/500000 hypothesis: tensor([18020.5430, 14040.2852, 18071.4746,  ..., 15837.0557, 16036.4561,
        12470.9375]) Cost: 6264.375000
Epoch 40000/500000 hypothesis: tensor([18046.3574, 13818.4043, 18097.2930,  ..., 15859.2510, 16053.8809,
        12205.6436]) Cost: 6233.813477
Epoch 50000/500000 hypothesis: tensor([18071.8359, 13575.0156, 18123.3828,  ..., 15855.5176, 16048.0664,
        11904.6885]) Cost: 6206.806641
Epoch 60000/500000 hypothesis: tensor([18105.8203, 13391.3730, 18157.3086,  ..., 15888.7783, 16076.6074,
        11683.8594]) Cost: 6183.051758
Epoch 70000/500000 hypothesis: tensor([18143.316

In [13]:
test_re = x_test.matmul(W) + b
test_re

tensor([[17801.4062],
        [13051.9678],
        [16929.8672],
        ...,
        [13649.0752],
        [17362.7324],
        [10284.7773]], grad_fn=<AddBackward0>)

In [14]:
submit['price'] = test_re.detach().numpy()
submission=submit.astype(np.int32)
submission.to_csv('submission.csv',mode='w',header=True,index=False)